Вариант 4: Дан код на языке Python, выравнивание в котором выполнено при помощи пробелов, все операторы записаны в одну строку, классы не используются. Представьте этот код в виде дерева по следующему алгоритму. Корнем дерева является фиктивная вершина. Все строки кода с одинаковым отступом являются вершинами одного уровня. Родителем строки является последняя перед ней строка с уровнем отступа на один меньше, при этом считается не число пробелов, а факт увеличения их количества. Потомки идут в том порядке, в котором они записаны в файле.
По заданному имени функции проверьте, является ли данная функция рекурсивной. В общем случае, условные конструкции, циклы и проч. не могут препятствовать вызову функции.

In [1]:
example = 'import random, math\ndef random_mult(my_list):\n    summa = 0\n    for i in range(30):\n        summa += math.log(my_list[i])\n    return math.exp(summa)'

In [30]:
def spaceCounter(code): #считаем отступы строчек кода
    code_dict = {}
    code_list = code.split('\n')
    for line in code_list:
        space_counter = 0
        if line[0] == ' ':
            for i in range(len(line) - 1):
                if line[i] == ' ':
                    space_counter += 1
                else:
                    break
        code_dict[line] = space_counter
    return code_dict            

In [60]:
spaceCounter(example)

{'import random, math': 0,
 'def random_mult(my_list):': 0,
 '    summa = 0': 4,
 '    for i in range(30):': 4,
 '        summa += math.log(my_list[i])': 8,
 '    return math.exp(summa)': 4}

In [59]:
code_dict = spaceCounter(example)

In [87]:
def makeATree(code_dict):
    tree = {'root': (-1, [])} #-1 - отступы, а в списке пишем потомков
    
    numbers = {} #номера строчек
    i = 0
    for elem in list(code_dict.keys()):
        numbers[elem] = i
        i += 1
    numbers['root'] = -1

    for line in code_dict: #создаём вершины
        tree[line] = (code_dict[line], [])
        
    values = reversed(sorted(set(code_dict.values()))) #упорядоченный список длин отступов
    
    for v in values:
        for son in code_dict:
            if code_dict[son] == v:
                best_mother = 'root'
                
                #ищем предка
                for mother in code_dict.keys():
                    if code_dict[mother] < code_dict[son]: #у предка меньше отступов
                        if numbers[mother] < numbers[son]: #предок раньше потомка
                            if numbers[mother] > numbers[best_mother]: #из возможных предков выбираем последнего
                                best_mother = mother               
                tree[best_mother][1].append(son) #предку записываем потомков
    return tree

In [88]:
makeATree(code_dict)

{'root': (-1, ['import random, math', 'def random_mult(my_list):']),
 'import random, math': (0, []),
 'def random_mult(my_list):': (0,
  ['    summa = 0', '    for i in range(30):', '    return math.exp(summa)']),
 '    summa = 0': (4, []),
 '    for i in range(30):': (4, ['        summa += math.log(my_list[i])']),
 '        summa += math.log(my_list[i])': (8, []),
 '    return math.exp(summa)': (4, [])}

In [89]:
tree = makeATree(code_dict)

In [151]:
def recursionTest(tree, name):
    occurrences = []
    for dot in tree:
        if name in dot:
            occurrences.append(dot) #собрали где встречается имя функции 
    if len(occurrences) < 2: 
        return False #если такой функции в коде нет, или если она вызывается только 1 раз, то она точно не рекурсивная
    elif 'def' not in occurrences[0]:
        return False #если у функции не было def в самом начале, то это и не функция, а какой-то другой текст
    else:
        return Test(tree, name, occurrences[0]) #танцуем от первой встречи функции - а она, естественно, там же, где def
        
def Test(tree, name, dot):
    #чтобы сказать, что функция рекурсивная, нам нужно, чтобы она была вызвана внутри самого своего описания (блока с def)
    #надо проверить её потомков
    for son in tree[dot][1]:
        if name in son:
            print('ура рекурсия')
            return True
        elif len(tree[son][1]) > 0: #если у потомка есть потомки
            Test(tree, name, son) #теперь танцуем от потомка
    return False #если ничего подобного не нашли

In [152]:
recursionTest(tree, 'random_mult')

False

In [138]:
recursionTest(tree, 'math')

False

In [109]:
#для примера рекурсивной функции возьмём факториал
r_code = 'def fac(n):\n    if n == 0:\n        return 1\n    return fac(n-1) * n\nprint(fac(5))'

In [110]:
r_dict = spaceCounter(r_code)
r_tree = makeATree(r_dict)

In [111]:
r_tree

{'root': (-1, ['def fac(n):', 'print(fac(5))']),
 'def fac(n):': (0, ['    if n == 0:', '    return fac(n-1) * n']),
 '    if n == 0:': (4, ['        return 1']),
 '        return 1': (8, []),
 '    return fac(n-1) * n': (4, []),
 'print(fac(5))': (0, [])}

In [136]:
recursionTest(r_tree, 'fac')

ура рекурсия


True

In [153]:
recursionTest(r_tree, 'return')

False

У меня ещё осталось время, поэтому напишу ещё функцию, как бы нам код из файла прочитать в строку и убрать лишние \n (если вдруг оно нужно).

In [149]:
import re

def codeReading(filename):
    with open(filename, encoding='utf-8') as f:
        code = f.read()
        code = re.sub(r'\n{2,}', '\n', code)
        return code